### Basic trading

- entry: do I use only the data for that stock endogenous, or also include the data from other stocks, exagenous.
- entry and exit :
- exit : take profit, stop loss, time out

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr
yf.pdr_override()

In [54]:
def read_data(stock='TSLA',short = 7,long = 20):


    df = pdr.get_data_yahoo(stock,period = "60d",

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = "1d",)


    emasUsed = [short,long]
    for ema in emasUsed:
        df['Ema_' + str(ema)] = round(df['Adj Close'].ewm(span = ema, adjust = False).mean(),2)
    df['Middle Band'] =df['Adj Close'].rolling(window=20).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Close'] > df['Upper Band'],'above_ballinger','normal')
    
    return df

In [55]:
def print_results(percentchange):
    gains = 0
    ng = 0
    losses = 0
    nl = 0
    totallR = 1
    for i in percentchange:
        if i >0:
            gains +=i
            ng +=1
        else:
            losses +=i
            nl +=1
        totallR = totallR*((i/100)+1)
    totallR = round((totallR-1)*100)
    if ng > 0:
        avgGain = gains/ng
        maxR = str(max(percentchange))
    else:
        avgGain = 0
        maxR = 'undefined'

    if nl>0:
        avgLoss = losses/nl
        maxL = str(min(percentchange))
        ratio = str(-(avgGain/avgLoss))
    else:
        avgLoss = 0
        maxL = 'undefined'
        ratio = 'inf'

    if ng >0 and nl >0:
        bettingAvg = ng/(ng+nl)
    else:
        bettingAvg = 0


    print()
    print('Result for'+stock+"going back to "+str(df.index[0])+" Sample size: "+str(ng+nl)+"trades")
    #print('EMAs used : ',str(emasUsed))
    print('Batting Avg : '+str(bettingAvg))
    print('Gain/loss ratio: ' + ratio)
    print('Avg gain: ' + str(avgGain))
    print('Avg loss: '+ str(avgLoss))
    print('Max return: '+ str(maxR))
    print('Max loss: ' + str(maxL))
    print('Total return over '+str(ng+nl)+' trades: '+str(totallR)+'%')
    print()



In [73]:
stock = input('Enter stock tikcer symbol:')

short = int(input('Enter short moving avg'))
long = int(input('Enter long term movin avg'))


short_moving_avg = 'Ema_'+ str(short)
long_moving_avg = 'Ema_'+ str(long)
#### 1. STRATEGY CROSSING



strategies = [1,2,3,4]


for strategy in strategies:



    if strategy == 1:
        df = read_data(stock, short,long)
        pos = 0
        num = 0
        percentchange = []
        start_of_this_month = int((len(df.index[:])/60)*30)

        for i in df.index[start_of_this_month:]:

            cmin = df[short_moving_avg][i]
            cmax = df[long_moving_avg][i]
            close = df['Adj Close'][i]
            #if(status_lower=='below_ballinger'):
           # if (cmin>cmax) and (status_lower=='below_ballinger'):
            if (cmin>cmax):
               # print('red white blue')
                if pos ==0:
                    bp =close
                    pos=1
                    print('Buying now at'+ str(bp))
                    #print(i)


            elif(cmin<cmax):
                #print('blue white red')
                if pos ==1:
                    pos = 0
                    sp = close
                    print('Selling now at'+ str(sp))
                    #print(i)
                    pc = (sp/bp-1)*100
                    percentchange.append(pc)

            if num ==df['Adj Close'].count()-1 and pos==1:
                pos = 0
                sp = close
                print('Selling now at'+ str(sp))
                pc = (sp/bp-1)*100
                percentchange.append(pc)      
            num +=1
        print_results(percentchange)
    
    if strategy == 2:
        
        df = read_data(stock, short,long)
        pos = 0
        num = 0
        percentchange = []

        for i in df.index:
            cmin = df[short_moving_avg][i]
            cmax = df[long_moving_avg][i]
            status_upper = df['status_upper'][i]
            status_lower = df['status_lower'][i]


            close = df['Adj Close'][i]

            if(status_lower=='below_ballinger'):
               # print('red white blue')
                if pos ==0:
                    bp =close
                    pos=1
                    #print('Buying now at'+ str(bp))
                    #print(i)


            elif(status_upper=='upper_ballinger'):
                #print('blue white red')
                if pos ==1:
                    pos = 0
                    sp = close
                    #print('Selling now at'+ str(sp))
                    #print(i)
                    pc = (sp/bp-1)*100
                    percentchange.append(pc)

            if num ==df['Adj Close'].count()-1 and pos==1:
                pos = 0
                sp = close
                #print('Selling now at'+ str(sp))
                pc = (sp/bp-1)*100
                percentchange.append(pc)      
            num +=1
        print_results(percentchange)

    elif strategy == 3:
        df = read_data(stock, short,long)
        pos = 0
        num = 0
        percentchange = []

        for i in df.index:
            cmin = df[short_moving_avg][i]
            cmax = df[long_moving_avg][i]
            status_upper = df['status_upper'][i]

            close = df['Adj Close'][i]

            #if(status_lower=='below_ballinger'):
           # if (cmin>cmax) and (status_lower=='below_ballinger'):
            if (cmin>cmax):
               # print('red white blue')
                if pos ==0:
                    bp =close
                    pos=1
                    #print('Buying now at'+ str(bp))
                    #print(i)


            elif(status_upper=='upper_ballinger'):
                #print('blue white red')
                if pos ==1:
                    pos = 0
                    sp = close
                    #print('Selling now at'+ str(sp))
                    #print(i)
                    pc = (sp/bp-1)*100
                    percentchange.append(pc)

            if num ==df['Adj Close'].count()-1 and pos==1:
                pos = 0
                sp = close
                #print('Selling now at'+ str(sp))
                pc = (sp/bp-1)*100
                percentchange.append(pc)      
            num +=1
        print_results(percentchange)

        
    elif strategy == 4:
        df = read_data(stock, short,long)
        pos = 0
        num = 0
        percentchange = []

        for i in df.index:
            cmin = df[short_moving_avg][i]
            cmax = df[long_moving_avg][i]
            status_lower = df['status_lower'][i]

            close = df['Adj Close'][i]

            if(status_lower=='below_ballinger'):
           # if (cmin>cmax) and (status_lower=='below_ballinger'):
               # print('red white blue')
                if pos ==0:
                    bp =close
                    pos=1
                    #print('Buying now at'+ str(bp))
                    #print(i)


            elif(cmin<cmax):
                #print('blue white red')
                if pos ==1:
                    pos = 0
                    sp = close
                    #print('Selling now at'+ str(sp))
                    #print(i)
                    pc = (sp/bp-1)*100
                    percentchange.append(pc)

            if num ==df['Adj Close'].count()-1 and pos==1:
                pos = 0
                sp = close
                print('Selling now at'+ str(sp))
                pc = (sp/bp-1)*100
                percentchange.append(pc)      
            num +=1
        print_results(percentchange)


    #print(percentchange)



## print results




Enter stock tikcer symbol: TSLA
Enter short moving avg 7
Enter long term movin avg 200


[*********************100%***********************]  1 of 1 completed
Buying now at425.67999267578125
Selling now at388.0400085449219
Buying now at438.0899963378906
Selling now at408.5

Result forTSLAgoing back to 2020-08-21 00:00:00 Sample size: 2trades
Batting Avg : 0
Gain/loss ratio: 0.0
Avg gain: 0
Avg loss: -7.798319202486237
Max return: undefined
Max loss: -8.842319295830247
Total return over 2 trades: -15.0%

[*********************100%***********************]  1 of 1 completed

Result forTSLAgoing back to 2020-08-21 00:00:00 Sample size: 1trades
Batting Avg : 0
Gain/loss ratio: inf
Avg gain: 5.2726499857062015
Avg loss: 0
Max return: 5.2726499857062015
Max loss: undefined
Total return over 1 trades: 5.0%

[*********************100%***********************]  1 of 1 completed

Result forTSLAgoing back to 2020-08-21 00:00:00 Sample size: 1trades
Batting Avg : 0
Gain/loss ratio: 0.0
Avg gain: 0
Avg loss: -5.1398641415943525
Max return: undefined
Max loss: -5.1398641415943525
Total ret